# Case Study: Cement Compressive Strength Prediction

In this case study, you'll solve a machine learning problem from start to finish.

The previous notebook, "analyze toydata", deals with a similar problem and can serve as a guideline for this exercise. You may also want to have a look at the [cheat sheet](https://franziskahorn.de/mlws_resources/cheatsheet.pdf) for more ideas and a concise overview of the relevant steps when developing a machine learning solution in any data science project. 

Feel free to get creative! 

### The Task

You are a data scientist at a startup that wants to build an app to help cement plants produce cement with a certain target compressive strength (= the main quality parameter of cement). Producing cement with a given strength is difficult, because the real compressive strength of a cement sample can only be measured after 28 days, at which point it is far too late to change anything in the production process and the produced cement needs to potentially be discarded.

During the cement production, some measurements about the material composition are available as well as the fineness of the cement powder. If you manage to predict the cement strength correctly from these parameters, then the cement production can be adjusted by changing the settings of the cement mill to make the cement powder finer or coarser. The cement plant already knows that milling the cement 1 micron finer increases the compressive strength by about 1 MPa, however, without accurate predictions of the current strength, they don't know when to change the fineness and therefore always mill each cement type with a fixed fineness that worked well on average in the past.

Your colleagues have already prepared the backend for the app (see notebook `b`), which will load your model to make the predictions. Additionally, the backend also contains code to optimize the cement's fineness to get closer to the specified target strength.

**Your job is it to build an ML model that predicts a cement sample's compressive strength.**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import joblib
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

# these "magic commands" are helpful if you plan to import functions from another script
# where you keep changing things, i.e., if you change a function in the script
# it will automagically be reloaded in the notebook so you work with the latest version
%load_ext autoreload
%autoreload 2

## Step 0: Load the data

The data contains samples from 5 different cement types, collected every Monday, Wednesday, and Friday. The data includes the compressive strength after 28 days as well as the fineness and various measurements of the material composition. Our goal is to **predict the compressive strength** of a cement sample based on the other values.

In [ ]:
# load the file into a dataframe with pandas
df = pd.read_csv("../data/cement_samples.csv")
# transform the timestamp into proper datetimes (for plotting)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.set_index("timestamp").sort_index()
# look at the raw data (first 5 rows)
df.head()

In [ ]:
# define features (all measurements known initially) and target (strength known only after 28 days)
# CAUTION: if you change these, you also need to adapt the feature list in the backend!
FEATURES = ["fineness"] + [f"material_{i}" for i in range(1, 16)]
TARGET = "strength"

In [ ]:
# these are the target strengths for each cement type
target_strengths = {
    "CEM I 52": 59.5,
    "CEM I 42": 50.5,
    "CEM II AS 52": 57.5,
    "CEM II AS 42": 50.,
    "CEM II BLL 32": 40.,
}

## Step 1: Exploratory Analysis

Get a better understanding of the data and the problem:
- How are the individual variables distributed?
- What differences can you observe between the different cement types?
- Does the data contain outliers?
- Can you observe any drifts in the values over time?
- Are any variables correlated (especially with the target)?
- Anything else you should take into account when preprocessing the data later for the supervised learning task?

In [ ]:
# TODO: create some plots to better understand the data


## Step 2: Predict the 28-day Compressive Strength

Now that you've become more familiar with the dataset, it's time to tackle the real task, i.e., predict the 28-day compressive strength of a cement sample.

An evaluation pipeline is already set up below using a "stupid baseline" (= predicting the mean). Your task is to improve upon the performance by trying, for exampe... 
- different [models](https://scikit-learn.org/stable/supervised_learning.html)
- different [preprocessing steps](https://scikit-learn.org/stable/modules/preprocessing.html) (e.g., transformations or feature engineering)
- [hyperparameter tuning](https://scikit-learn.org/stable/modules/grid_search.html)
- [ensemble models](https://scikit-learn.org/stable/modules/ensemble.html) (i.e., combining different models)

Get creative :-)

**Tip:** To use your model within the app's backend later (notebook `b`), it's important that your final model incl. all necessary preprocessing steps are combined in a single estimator object. This can be accomplished with sklearn's [`make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html) function. If necessary, you could even write a [custom transformer](https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-ef792bbb3260/) to perform more fancy feature engineering steps than what is provided by sklearn.


In [ ]:
# extract a training and final test set from the full dataframe
# since the samples are collected over time, we use the most recent data for testing
# we also leave a 28 day gap between train and test to avoid data leakage
df_train, df_test = df[df.index < "2024-12-03"], df[df.index >= "2025-01-01"]

In [ ]:
def eval_model(model, df_train):
    """
    Function to evaluate a regressor using a timeseries cross-validation.
    Prints mean absolute error, averaged across xval folds.
    
    The best possible MAE (if you had access to some hidden information) is 0.3 (due to measurement noise).
    A realistic good MAE that you should try to achieve is 0.52
    
    Inputs:
        - model: the initialized model
        - df_train: the training data
    """
    # to make it realistic, we need to leave a 28 day gap - that is 3 samples * 4 weeks * number of cements
    gap = 12 * df_train["cement"].nunique()
    tscv = TimeSeriesSplit(gap=gap, n_splits=10)
    res = cross_val_score(model, df_train[FEATURES], df_train[TARGET], scoring="neg_mean_absolute_error", cv=tscv)
    print(f"Cross-validated MAE: {np.mean(np.abs(res)):.2f}")

In [ ]:
# initialize a dummy model
model = DummyRegressor()
# evaluate the model
eval_model(model, df_train)

In [ ]:
# TODO: now it's up to you: try an actual model and get better predictions!


In [ ]:
# train your final model on the full training dataset
model = model.fit(df_train[FEATURES], df_train[TARGET])
# and save it so it can be used by the backend
# CAUTION: your model must be a single estimator object that also includes all necessary preprocessing steps 
# (e.g., by using the `make_pipeline` function mentioned above) which works on the originally defined features
joblib.dump(model, "strength_model.pkl")

## Step 3: Interpret the model

After you've found a model that achieves a decent performance, try to understand what it's doing.
- Calculate the permutation feature importance to see which features are most influential overall
- For the most important features, look at the partial dependence plot to see _how_ these features influence the outcome

Do these results make sense given what you know about the data?

In [ ]:
# TODO: permutation feature importance


In [ ]:
# TODO: partial dependence plots


## Step 4: Use the backend

Head over to notebook `b` and run it from top to bottom and leave it open. It runs the server with our backend using your trained and saved model to make predictions.

In the code below we query the backend with a single data point to see what it returns.

In [ ]:
# url where the backend is running
port = 8000
backend_url = f"http://localhost:{port}/predict"
# columns that the backend expects
JSON_COLS = ["cement"] + FEATURES

In [ ]:
# get one row from our dataframe and convert it to a dictionary
x = df_test[JSON_COLS].iloc[0].to_dict()
# add the target strength
x["target_strength"] = target_strengths[x["cement"]]
# send this as a json to the backend via a POST request
response = requests.post(backend_url, json=x)
# the result will also be dictionary with
# - the original fineness
# - the predicted strength with the original fineness
# - the optimized fineness
# - the new predicted strength when using the optimized fineness
result = response.json()
result

## Step 5: Optimization & What-If Analysis

When producing cement, the quality of the raw materials is fixed (whatever we dig up in the quarry) but the fineness can be adjusted in the mill to compensate for quality differences and change the resulting compressive strength of the cement in accordance with this cement's target strength.

Your prediction model is used in the backend to check whether the cement would get too strong or too weak and then the fineness is adapted accordingly to get closer to the target.

Does your model help to get the production on target?

In [ ]:
def plot_what_if(df_cem, target_strength):
    """
    Compute and plot the optimized the fineness for all samples from one cement.
    This code creates two plots:
        1. What-if results after optimization: by changing the fineness, the strength predictions
           should be closer to the target strength, even after correcting for prediction errors.
           The legend includes the MATD = mean absolute target deviation, i.e., how far away the
           respective points are from the target strength on average.
        2. Original and optimized fineness and resulting strength increase/decrease.
    
    Inputs:
        - df_cem: pandas dataframe with the data from one cement
        - target_strength: what we would like the compressive strength to be
    """
    # copy data so we don't change the original
    df = df_cem.copy()
    # run the optimization for all data points
    fineness_org_s, fineness_new_s, pred_org_s, pred_new_s = [], [], [], []
    for i in range(len(df)):
        if not i % 10:
            print(f"backend queried for {i:2} / {len(df)} data points", end="\r")
        x = df[JSON_COLS].iloc[i].to_dict()
        x["target_strength"] = target_strength
        result = requests.post(backend_url, json=x).json()
        fineness_org_s.append(result["fineness_org"])
        fineness_new_s.append(result["fineness_new"])
        pred_org_s.append(result["pred_org"])
        pred_new_s.append(result["pred_new"])
    print(f"backend queried for {len(df)} / {len(df)} data points")
    # add results to dataframe for easier plotting
    df["fineness_org"] = fineness_org_s
    df["fineness_new"] = fineness_new_s
    df["pred_org"] = pred_org_s
    df["pred_new"] = pred_new_s
    
    # plot the optimization results
    plt.figure(figsize=(10, 5))
    # dashed line to indicated the target value we wanted
    plt.hlines(target_strength, df.index.min(), df.index.max(), "k", "dashed", linewidth=1)
    # original strength values as light blue dots
    plt.plot(df[TARGET], "o", c="#53DDFE", alpha=0.8, 
             label=f"original y (MATD: {np.abs(df[TARGET] - target_strength).mean():.1f})")
    # predicted strength values with original fineness as blue x
    plt.plot(df["pred_org"], "x", c="#007693", 
             label=f"predicted y (MATD: {np.abs(df["pred_org"] - target_strength).mean():.1f})")
    # predicted strength values with optimized fineness as orange x
    plt.plot(df["pred_new"], "x", c="#A84801", 
             label=f"optimized predicted y (MATD: {np.abs(df["pred_new"] - target_strength).mean():.1f})")
    # since our original predictions are not perfect, we shift our optimized predictions
    # by the error we made on the original predictions -> plot as orange dots
    pred_new_corrected_s = df["pred_new"] + (df[TARGET] - df["pred_org"])
    plt.plot(pred_new_corrected_s, "o", alpha=0.8, c="#FE9C53", 
             label=f"realistic optimized y (MATD: {np.abs(pred_new_corrected_s - target_strength).mean():.1f})")
    plt.xlabel("samples")
    plt.ylabel("compressive strength [MPa]")
    plt.title(f"What-If Analysis for {df['cement'].iloc[0]}")
    plt.legend(loc=2, bbox_to_anchor=(1.02, 1), numpoints=1)
    
    # plot original and optimized fineness
    plt.figure()
    # diagonal line -> original and optimized fineness are the same
    min_max = [df["fineness_new"].min(), df["fineness_new"].max()]
    plt.plot(min_max, min_max, "k", alpha=0.5)
    # points above the line: finer than before
    # points below the line: coarser than before
    # color of dot shows whether the optimization resulted in a reduction or increase in predicted strength
    plt.scatter(df["fineness_org"], df["fineness_new"], c=df["pred_new"]-df["pred_org"])
    plt.colorbar()
    plt.xlabel("original fineness")
    plt.ylabel("optimized fineness")
    plt.title("changes in fineness & strength");
    

In [ ]:
# run the optimization on the test set and plot the results for one cement
cement = "CEM II AS 42"
plot_what_if(df_test[df_test["cement"] == cement], target_strengths[cement])

## Step 6: Presentation of results
Clean up your code & think about which results you want to present and the story they tell:
- What have you learned about cement and how is this reflected in the data?
- What is the best model that you found & its performance?
- Which preprocessing steps had the most impact on the performance (for different models)?
- Which features were the most influential and how did they impact the model's prediction?
- What have you learned in this case study? Did any of the results surprise you?